<a href="https://colab.research.google.com/github/AUT-Student/NLP-HW3/blob/main/NLP_HW3_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: Natural Language Processing
<br>
<b>Description:</b> HomeWork 3 | Question 1
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# Libraries

In [1]:
!pip install bert_embedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.6 MB 96.5 MB/s 
     |████████████████████████████████| 209 kB 59.7 MB/s 
     |████████████████████████████████| 13.8 MB 39.7 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-py3-none-any.whl size=259930 sha256=6bf5192959dc5d7c3e55a703c2f8c2798a7d3312b2034ed977fe86005c9fa33e
  Stored in directory: /root/.cache/pip/wheels/a6/41/8f/45bd1c58055d87aee5a71b6756a427ea8d92e506b3a9d17370
Successfully built gluonnlp
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [60]:
import pandas as pd
import numpy as np
from types import SimpleNamespace
import prettytable
from prettytable import PrettyTable
from bert_embedding import BertEmbedding
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# Dataset 

In [3]:
!gdown 1oin_Sw1Gk_WLS9zpDrap5FdlRcUrTP_D
!gdown 1EvtGQ8-sYXXQ3VA9ByjD4OmFH13WxvAs
!gdown 154f-z0PsPAp0yvOLdNXP8NXYjPljpgCZ
!gdown 1EDG_j6F5ohIjpkihRQd0-i9daHWV7StN

Downloading...
From: https://drive.google.com/uc?id=1oin_Sw1Gk_WLS9zpDrap5FdlRcUrTP_D
To: /content/Sentenses_train.txt
100% 1.23M/1.23M [00:00<00:00, 125MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EvtGQ8-sYXXQ3VA9ByjD4OmFH13WxvAs
To: /content/Senses_train.txt
100% 49.7k/49.7k [00:00<00:00, 46.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=154f-z0PsPAp0yvOLdNXP8NXYjPljpgCZ
To: /content/Sentenses_test.txt
100% 213k/213k [00:00<00:00, 86.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EDG_j6F5ohIjpkihRQd0-i9daHWV7StN
To: /content/Senses_test.txt
100% 8.76k/8.76k [00:00<00:00, 10.4MB/s]


In [4]:
def create_dataset(part_name):
  sentences_file = open(f"/content/Sentenses_{part_name}.txt", "r")
  senses_file = open(f"/content/Senses_{part_name}.txt", "r")

  sentences = sentences_file.readlines()
  senses = senses_file.readlines()
  
  dataset_sentence = []
  dataset_sense = []
  dataset_ambiguity_word = []

  for sentence, sense in list(zip(sentences, senses)):
    dataset_sense.append(sense[:-1])
    tokens = sentence.split()
    word_index = tokens.index("<head>")
    dataset_ambiguity_word.append(tokens[word_index+1].lower())
    dataset_sentence.append(" ".join(
        tokens[max(0, word_index-11):word_index-1] +\
        [tokens[word_index+1]] +\
        tokens[word_index+3:min(word_index+13, len(tokens))])
    )
  
  return pd.DataFrame({"Sentence": dataset_sentence, "Ambiguity_Word": dataset_ambiguity_word, "Sense": dataset_sense})

In [5]:
train_dataset = create_dataset("train")
test_dataset = create_dataset("test")

# Embedding

In [6]:
# GPU
# ctx = mx.gpu(0)
# bert_embedding = BertEmbedding(ctx=ctx)

# CPU
bert_embedding = BertEmbedding()

Vocab file is not found. Downloading.


In [7]:
def calculate_ambiguity_vector(dataset):
  dataset["Ambiguity_Vector"] = None
  for i, data in dataset.iterrows():
    if i%300==0: print(f"{int(i*100/len(dataset))}%")
    tokens, vectors = bert_embedding([data["Sentence"]])[0]
    ambiguity_word = data["Ambiguity_Word"]

    try:    
      ambiguity_vector = vectors[tokens.index(ambiguity_word)]
    except ValueError:
      tokens, vectors = bert_embedding([data["Sentence"][15:]])[0]
      ambiguity_word = data["Ambiguity_Word"]
      ambiguity_vector = vectors[tokens.index(ambiguity_word)]

    dataset.at[i, "Ambiguity_Vector"] = ambiguity_vector

In [8]:
calculate_ambiguity_vector(train_dataset)

0%
4%
9%
14%
18%
23%
28%
33%
37%
42%
47%
52%
56%
61%
66%
71%
75%
80%
85%
90%
94%
99%


In [9]:
calculate_ambiguity_vector(test_dataset)

0%
26%
53%
80%


# Dimension Reduction

In [10]:
pca = PCA(n_components=300)

In [11]:
pca.fit(np.array([item for item in train_dataset["Ambiguity_Vector"].values]))

PCA(n_components=300)

In [12]:
pca.transform(np.array([item for item in train_dataset["Ambiguity_Vector"].values])).shape

(6324, 300)

In [13]:
train_dataset["Ambiguity_Vector_Reduced"] = [item for item in pca.transform(np.array([item for item in train_dataset["Ambiguity_Vector"].values]))]

In [14]:
test_dataset["Ambiguity_Vector_Reduced"] = [item for item in pca.transform(np.array([item for item in test_dataset["Ambiguity_Vector"].values]))]

# Train Classifiers

In [16]:
set(train_dataset["Sense"].values)

{'HARD1',
 'HARD2',
 'HARD3',
 'SERVE10',
 'SERVE12',
 'cord',
 'division',
 'formation',
 'interest1',
 'interest2',
 'interest3',
 'interest4',
 'interest5',
 'interest6',
 'phone',
 'product',
 'text'}

In [17]:
set(train_dataset["Ambiguity_Word"].values)

{'hard',
 'harder',
 'hardest',
 'interest',
 'interested',
 'interesting',
 'interests',
 'line',
 'lines',
 'serve',
 'served',
 'serves'}

In [18]:
hard_words = ["hard", "harder", "hardest"]
interest_words = ["interest", "interested", "interesting", "interests"]
line_words = ["line", "lines"]
serve_words = ["serve", "served", "serves"]

hard_senses = ["HARD1", "HARD2", "HARD3"]
interest_senses = ["interest1", "interest2", "interest3", "interest4", "interest5", "interest6"]
line_senses = ["cord", "division", "formation", "phone", "product", "text"]
serve_senses = ["SERVE10", "SERVE12"]

mapping_word_sense = {
    "HARD": (hard_words, hard_senses),
    "INTEREST": (interest_words, interest_senses),
    "LINE": (line_words, line_senses),
    "SERVE": (serve_words, serve_senses)
}

In [49]:
valid_dataset = train_dataset.sample(frac=0.2, random_state=0)
train_dataset = train_dataset.drop(valid_dataset.index)

In [68]:
def report_accuracy_f(classifier, dataset, verbose=False):
  y_predict = classifier.predict(np.stack(dataset["Ambiguity_Vector_Reduced"].values))
  y_true = dataset["Sense"].values

  accuracy = round(accuracy_score(y_true, y_predict)*100, 2)
  f1 = round(f1_score(y_true, y_predict, average='macro')*100, 2)

  if verbose:
    print(f"Accuracy = {accuracy}")
    print(f"F1 = {f1}")  
  
  return accuracy, f1

In [69]:
classifiers = {}

settings = [
            {"kernel": "linear"},
            {"kernel": "poly", "degree":3},
            {"kernel": "poly", "degree":6},
            {"kernel": "rbf"}
]

for key in mapping_word_sense.keys():
  words = mapping_word_sense[key][0]
  senses = mapping_word_sense[key][1]

  sub_train_dataset = train_dataset[train_dataset["Ambiguity_Word"].isin(words) & train_dataset["Sense"].isin(senses)]
  
  sub_valid_dataset = valid_dataset[valid_dataset["Ambiguity_Word"].isin(words) & valid_dataset["Sense"].isin(senses)]

  if key == "HARD":      
    classifier = SimpleNamespace(predict=lambda x:np.array(["HARD1"]*len(x)))

    classifiers[key] = {"classifier": classifier, "setting": None}  
  else:

    best_accuracy = 0
    best_classifier = None
    best_setting = None

    for setting in settings:
      classifier = SVC(**setting)
      classifier.fit(X=np.stack(sub_train_dataset["Ambiguity_Vector_Reduced"].values), y=sub_train_dataset["Sense"].values)
      
      accuracy = report_accuracy_f(classifier, sub_valid_dataset)[0]

      if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_classifier = classifier
        best_setting = setting

    classifiers[key] = {"classifier": best_classifier, "setting": best_setting}

In [74]:
table = PrettyTable()
table.field_names = ["Word", "Setting", "Train Number", "Valid Number", "Test Number",
                     "Train Accuracy", "Valid Accuracy", "Test Accuracy",
                     "Train F1", "Valid F1", "Test F1"]

sum_train_number, sum_valid_number, sum_test_number = 0, 0, 0
sum_train_accuracy, sum_valid_accuracy, sum_test_accuracy= 0, 0, 0
sum_train_f1, sum_valid_f1, sum_test_f1 = 0, 0, 0

for key in mapping_word_sense.keys():
  words = mapping_word_sense[key][0]
  senses = mapping_word_sense[key][1]

  sub_train_dataset = train_dataset[train_dataset["Ambiguity_Word"].isin(words) & train_dataset["Sense"].isin(senses)]
  sub_valid_dataset = valid_dataset[valid_dataset["Ambiguity_Word"].isin(words) & valid_dataset["Sense"].isin(senses)]
  sub_test_dataset = test_dataset[test_dataset["Ambiguity_Word"].isin(words) & test_dataset["Sense"].isin(senses)]
  
  classifier = classifiers[key]["classifier"]
  setting = classifiers[key]["setting"]

  train_number = len(sub_train_dataset)
  valid_number = len(sub_valid_dataset)
  test_number = len(sub_test_dataset)

  sum_train_number += train_number
  sum_valid_number += valid_number
  sum_test_number += test_number

  train_accuracy, train_f1 = report_accuracy_f(classifier, sub_train_dataset)
  valid_accuracy, valid_f1 = report_accuracy_f(classifier, sub_valid_dataset)
  test_accuracy, test_f1 = report_accuracy_f(classifier, sub_test_dataset)

  sum_train_accuracy += train_accuracy * train_number
  sum_valid_accuracy += valid_accuracy * valid_number
  sum_test_accuracy += test_accuracy * test_number
  
  sum_train_f1 += train_f1 * train_number
  sum_valid_f1 += valid_f1 * valid_number
  sum_test_f1 += test_f1 * test_number
  
  table.add_row([key, setting, train_number, valid_number, test_number,
                 train_accuracy, valid_accuracy, test_accuracy,
                 train_f1, valid_f1, test_f1])

avg_train_accuracy = round(sum_train_accuracy/sum_train_number, 2)
avg_valid_accuracy = round(sum_valid_accuracy/sum_valid_number, 2)
avg_test_accuracy = round(sum_test_accuracy/sum_test_number, 2)

avg_train_f1 = round(sum_train_f1/sum_train_number, 2)
avg_valid_f1 = round(sum_valid_f1/sum_valid_number, 2)
avg_test_f1 = round(sum_test_f1/sum_test_number, 2)

table.add_row(["Overall", None, sum_train_number, sum_valid_number, sum_test_number,
               avg_train_accuracy, avg_valid_accuracy, avg_test_accuracy,
               avg_train_f1, avg_valid_f1, avg_test_f1
              ])

print(table)

+----------+----------------------+--------------+--------------+-------------+----------------+----------------+---------------+----------+----------+---------+
|   Word   |       Setting        | Train Number | Valid Number | Test Number | Train Accuracy | Valid Accuracy | Test Accuracy | Train F1 | Valid F1 | Test F1 |
+----------+----------------------+--------------+--------------+-------------+----------------+----------------+---------------+----------+----------+---------+
|   HARD   |         None         |     1334     |     327      |     300     |     100.0      |     100.0      |     100.0     |  100.0   |  100.0   |  100.0  |
| INTEREST | {'kernel': 'linear'} |     1343     |     337      |     285     |     100.0      |     91.69      |     89.82     |  100.0   |  62.96   |  74.61  |
|   LINE   |  {'kernel': 'rbf'}   |     1185     |     270      |     236     |     97.38      |     92.59      |     94.49     |   97.2   |  92.74   |  94.09  |
|  SERVE   | {'kernel': 'lin